In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
text="C:/Users/levent/OneDrive/Masaüstü/Doğal+Dil+İşleme+Kodlar/Hakan Cebeci - Doğal Dil İşleme/4- Sentiment Analysis/hepsiburada.csv"

In [3]:
dataset= pd.read_csv(text)

In [4]:
dataset

,Rating,Review
0,1,3 yıldır tık demedi. :)
1,1,3 yıldır kullanıyorum müthiş
2,1,Ürün bugün elime geçti çok fazla inceleme fırs...
3,1,Almaya karar verdim. Hemencecik geldi. Keyifle...
4,1,Günlük kullanımınızı çok çok iyi karsılıyor kı...
...,...,...
243492,1,fiyatına göre güzel
243493,1,Ürün kullanışlı iş görüyor fazlasıyla eşime al...
243494,1,"Hızlı Kargo, güzel ürün"
243495,1,telefon başarılı hızlı bir cihaz sadece beyaz...


In [5]:
#Dataframe formatındaki verilerin trating ve Review kolonları alındı ve liste formatına dönüştürüldü

target = dataset['Rating'].values.tolist()
data = dataset['Review'].values.tolist()

In [6]:
cutoff=int(len(dataset)*0.8)

In [7]:
#veriler %80 oranında train, %20 oranında test olarak ayrıldı ve bu ayrılma hem data hemde etiket kolonlarına uygulandı.
x_train, x_test = data[:cutoff], data[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]

In [8]:
x_train[15]

'Hoş ve kaliteli bir mouse. Leptopla kullanım için mükemmel boyutlarda. Umarım uzun süre bozulmaz.'

In [9]:
#Oluşturulacak tokenleştirme işleminde en çok kullanılan 10000 kelimenin kullanılmasını istedik.
num_words = 10000

In [10]:
tokenizer = Tokenizer(num_words=num_words)

In [11]:
#Oluşturulan tokenizer nesnesi ile yorumlar tokenlere ayrıldı
tokenizer.fit_on_texts(data)

In [12]:
tokenizer.word_index

{'çok': 1,
 'bir': 2,
 've': 3,
 'ürün': 4,
 'bu': 5,
 'iyi': 6,
 'güzel': 7,
 'için': 8,
 'tavsiye': 9,
 'ederim': 10,
 'daha': 11,
 'ama': 12,
 'da': 13,
 'gayet': 14,
 'hızlı': 15,
 'teşekkürler': 16,
 'aldım': 17,
 'de': 18,
 'ürünü': 19,
 'gibi': 20,
 'yok': 21,
 'uygun': 22,
 'olarak': 23,
 'kaliteli': 24,
 'en': 25,
 '2': 26,
 'kargo': 27,
 'fiyat': 28,
 'elime': 29,
 'kadar': 30,
 'ile': 31,
 'göre': 32,
 'geldi': 33,
 'var': 34,
 'hepsiburada': 35,
 'ben': 36,
 'gerçekten': 37,
 '1': 38,
 'fiyata': 39,
 'gün': 40,
 'sonra': 41,
 'cok': 42,
 'kesinlikle': 43,
 'telefon': 44,
 'biraz': 45,
 'hiç': 46,
 'ulaştı': 47,
 'memnun': 48,
 'hem': 49,
 'değil': 50,
 'kullanışlı': 51,
 '3': 52,
 'mükemmel': 53,
 'oldu': 54,
 'kullanıyorum': 55,
 'önce': 56,
 'sipariş': 57,
 'tek': 58,
 'her': 59,
 'bence': 60,
 'harika': 61,
 'kalitesi': 62,
 'bi': 63,
 'ayrıca': 64,
 '5': 65,
 'teşekkür': 66,
 'fiyatı': 67,
 'olması': 68,
 'ne': 69,
 'herkese': 70,
 'bile': 71,
 'uzun': 72,
 'süper': 73,

In [13]:
#texts_to_sequences fonksiyonu ile tokenlerin yerini index değerleri aldı ve tokenler sayısallaştırılmış oldu.
x_train_sequens = tokenizer.texts_to_sequences(x_train)

In [14]:
print(x_train_sequens[800])

[19, 341, 52, 194, 30, 54, 7992, 55, 209, 603, 7887, 3, 36, 114, 164, 479, 85, 1, 1682, 782, 30, 2, 380, 326, 3, 1, 81, 132, 562, 9, 10]


In [15]:
#aynı sayısallaştırma işlemine X_test verilerine de tabi tutuldu.
x_test_sequens = tokenizer.texts_to_sequences(x_test)

In [16]:
print(x_test_sequens[800])

[3226, 308, 2, 4699, 260, 693, 185, 1556, 1810, 185, 64, 3379, 20, 78, 30, 13, 269, 2, 550, 50]


In [17]:
#Verilerin modele verilmeye hazır hale getirilmesi için tüm tokenlerin uzunluğunun eşit olması gerekir.Bunun için tüm x verileri bir listeye atandı
#ve uzunlukları alındı.
num_token =[len(token) for token in x_train_sequens + x_test_sequens ]
num_token= np.array(num_token)

In [18]:
#Verilrin uzunuğunun ortalaması
np.mean(num_token)

20.744703220162876

In [19]:
#Verilrin uzunuğunun maximumu
np.max(num_token)

295

In [20]:
#argmax ile en büyük elemanın indisine ulaşılır
np.argmax(num_token)

21941

In [21]:
x_train[21941]

'Özellikle bu kısma yazıyorum iyice okuyunuz,cihazın hızı çok iyi.Isınma normal boyutlarda.Kamerası elinizi sabit tutarsanız ve gündüz çok net gece ise çok net çekmez.Görüntülü konuşma özelliği yok ancak uygulama ile olur,anten çekimi gayet iyi.Parmak izi okuyucusu gayet hızlı bazen tuşu silmenizi istiyor ve okuyamıyor kirden vs...Ön kamera da tatmin edici,çekim esnasında ekran beyaz ışık vererek flaş görevi görür.Batarya bana 1 hafta gidiyor sık kullanımda ise 2-3 gün gidiyor.Hızlı şarj 1 saatte doluyor , şekilleri ele oturuyor ve şık bir görüntü var.Telefonu aldığım gün gittim ve ilk girdiğim yerden ekran koruyucu ve kılıf buldum.Kulaklık sesi çok net ve yüksek ancak kendi hoparlörü biraz zayıf sesi.Ekrana bakarken açık kalma özelliği yok.Diğer akıllı cihaz özellikleri %90 ı bu cihazda mevcut.Güç tasarrufu 2 ayrı modu var ve çok başarılı çalışıyor.4gb ram var genelde yarısı boş kalıyor.Bir de yeni cihazların çoğu titreşimi az ve sesi de az çıkıyor.Bu cihaz da içine dahil...Bu cihazla

In [22]:
#Verilerin uzunluk ortalaması 20 olmasına rağmen biz ortalama+ 2x standart sapma formülünün çıktını uygulayacağız.
max_tokens = np.mean(num_token) + 2 * np.std(num_token)

In [23]:
max_tokens

59.81181704385857

In [24]:
max_tokens=int(max_tokens)

In [25]:
#Buradan da kullandığımız formülün çıktısı olan 59 un verilerin %95 i kapsadığını görebiliriz.
np.sum(num_token < max_tokens )/ len(num_token)

0.9597982726686571

In [26]:
x_train_pad=  pad_sequences(x_train_sequens, maxlen = 59)
x_test_pad=  pad_sequences(x_test_sequens ,maxlen = 59)

In [27]:
x_train_pad.shape

(194797, 59)

In [28]:
x_test_pad.shape

(48700, 59)

In [29]:
x_test_pad[800]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0, 3226,  308,    2, 4699,  260,
        693,  185, 1556, 1810,  185,   64, 3379,   20,   78,   30,   13,
        269,    2,  550,   50])

In [30]:
np.array(x_train_sequens[800])

array([  19,  341,   52,  194,   30,   54, 7992,   55,  209,  603, 7887,
          3,   36,  114,  164,  479,   85,    1, 1682,  782,   30,    2,
        380,  326,    3,    1,   81,  132,  562,    9,   10])

In [31]:
x_train_pad[800]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,   19,  341,   52,  194,   30,
         54, 7992,   55,  209,  603, 7887,    3,   36,  114,  164,  479,
         85,    1, 1682,  782,   30,    2,  380,  326,    3,    1,   81,
        132,  562,    9,   10])

In [32]:
#
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [33]:
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text

In [34]:
x_train[800]

'ürünü alalı 3 hafta kadar oldu. aralıksız kullanıyorum bilgisyarım sürekli açık durur ve ben günde yaklaşık 12 saat başındayım mousesu çok kullanırım. şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim.'

In [35]:
tokens_to_string(x_train_sequens[800])

'ürünü alalı 3 hafta kadar oldu aralıksız kullanıyorum sürekli açık durur ve ben günde yaklaşık 12 saat çok kullanırım şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim'

In [36]:
#Artık modelimizi oluşturmaya başlayabiliriz!
model=Sequential()
mbedding_size=50
model.add(Embedding(input_dim=num_words,
                   output_dim=50,
                   input_length=max_tokens,
                    name='embedding_layer'))

In [37]:
#Buradaki GRU yerine LSTM katmanları da kullanılabilri.
#Ayrıca  return_sequences parametresi ,eyer sonraki katman GRU VEYA LSTM kullanmıyorsa true olur.
#Eyer sonrasında output üretilecekse defult olarak false yapılır.

model.add(GRU(units=16,return_sequences=True)) 
model.add(GRU(units=8,return_sequences=True)) 
model.add(GRU(units=4)) 

model.add(Dense(1,activation='sigmoid'))

In [38]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_layer (Embedding)  (None, 59, 50)           500000    
                                                                 
 gru (GRU)                   (None, 59, 16)            3264      
                                                                 
 gru_1 (GRU)                 (None, 59, 8)             624       
                                                                 
 gru_2 (GRU)                 (None, 4)                 168       
                                                                 
 dense (Dense)               (None, 1)                 5         
                                                                 
Total params: 504,061
Trainable params: 504,061
Non-trainable params: 0
_________________________________________________________________


In [45]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

In [46]:
y_train = np.array(y_train)

In [47]:
model.fit(x_train_pad,y_train,
          epochs=5,
          batch_size=256
         )

Epoch 1/5
761/761 [==============================] - 19s 17ms/step - loss: 0.1623 - accuracy: 0.9548
Epoch 2/5
761/761 [==============================] - 13s 17ms/step - loss: 0.0888 - accuracy: 0.9709
Epoch 3/5
761/761 [==============================] - 13s 17ms/step - loss: 0.0724 - accuracy: 0.9769
Epoch 4/5
761/761 [==============================] - 13s 17ms/step - loss: 0.0596 - accuracy: 0.9817
Epoch 5/5
761/761 [==============================] - 14s 18ms/step - loss: 0.0482 - accuracy: 0.9862


In [49]:
y_test=np.array(y_test)

In [50]:
#Modelimizin doğruluk oranı
result= model.evaluate(x_test_pad,y_test)

1522/1522 [==============================] - 11s 7ms/step - loss: 0.1560 - accuracy: 0.9510


In [52]:
result

[0.15603867173194885, 0.9510266780853271]

In [54]:
#ilk 1000 değer test edildi
predict_results= model.predict(x=x_test_pad[:1000])

32/32 [==============================] - 1s 9ms/step


In [57]:
#Modelin düz satır halinde gelmesi için transpozu alındı
predict_results = predict_results.T[0]

In [58]:
predict_results

array([0.99705493, 0.99770606, 0.99560237, 0.95009226, 0.9975973 ,
       0.91090024, 0.9970138 , 0.9971475 , 0.9969861 , 0.99288964,
       0.99285156, 0.99604475, 0.9928381 , 0.99720144, 0.9958668 ,
       0.997186  , 0.9973998 , 0.9972869 , 0.996582  , 0.99774206,
       0.9976495 , 0.99616635, 0.99690634, 0.8965528 , 0.9932366 ,
       0.996912  , 0.98336095, 0.9946867 , 0.99186546, 0.9535528 ,
       0.9975508 , 0.50924057, 0.9905233 , 0.997652  , 0.05243561,
       0.9974256 , 0.995226  , 0.9976903 , 0.9974529 , 0.98112065,
       0.9976191 , 0.9958139 , 0.99721587, 0.9961868 , 0.9951362 ,
       0.9941016 , 0.9885661 , 0.99487853, 0.9950867 , 0.99774855,
       0.99768937, 0.9958995 , 0.9967405 , 0.9973189 , 0.9961247 ,
       0.9968118 , 0.99729615, 0.01328996, 0.9775729 , 0.9976388 ,
       0.997584  , 0.99747497, 0.9972874 , 0.9971517 , 0.9975866 ,
       0.99755955, 0.99276227, 0.2985303 , 0.9934683 , 0.98426914,
       0.9976956 , 0.9699775 , 0.99722856, 0.9941552 , 0.03194

In [60]:
#değerlerimizi 0.5 ten küçük veya büyük olma durumuna göre 0 ve 1 e eşitledik.
class_pred=np.array([1.0 if q>0.5 else 0.0 for q in  predict_results ])

In [61]:
class_pred

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0.

In [69]:
class_true=np.array(y_test[:1000])

In [70]:
class_true

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,

In [71]:
incorrect=  np.where(class_true!=class_pred)

In [86]:
dx=len(incorrect[0])

In [90]:
dx=incorrect[0]

In [92]:
dx[12]

162

In [96]:
text=x_test[dx[12]]

In [100]:
x_test[162]

'iki kedi de bile hiç koku yok'

In [98]:
tokens_to_string(x_test_sequens[162])

'iki kedi de bile hiç koku yok'

In [102]:
predict_results[162]

0.2836033

In [104]:
y_test[162]

1

In [105]:
text1 = "bu ürün çok iyi herkese tavsiye ederim"
text2 = "kargo çok hızlı aynı gün elime geçti"
text3 = "büyük bir hayal kırıklığı yaşadım bu ürün bu markaya yakışmamış"
text4 = "mükemmel"
text5 = "tasarımı harika ancak kargo çok geç geldi ve ürün açılmıştı tavsiye etmem"
text6 = "hiç resimde gösterildiği gibi değil"
text7 = "kötü yorumlar gözümü korkutmuştu ancak hiçbir sorun yaşamadım teşekkürler"
text8 = "hiç bu kadar kötü bir satıcıya denk gelmemiştim ürünü geri iade ediyorum"
text9 = "tam bir fiyat performans ürünü"
text10 = "beklediğim gibi çıkmadı"
texts = [text1, text2, text3, text4, text5, text6, text7, text8, text9, text10]

In [106]:
tokens= tokenizer.texts_to_sequences(texts)

In [107]:
tokens_pad= pad_sequences(tokens,maxlen=59)

In [109]:
last_results=model.predict(tokens_pad)

1/1 [==============================] - 0s 27ms/step


In [110]:
pred=np.array([1.0 if q>0.5 else 0.0 for q in last_results ])

In [111]:
pred

array([1., 1., 0., 1., 0., 0., 1., 0., 1., 0.])